# Trying to install the Python modules and Add github Project files to Colab Drive and enviroment

In [61]:
!git clone https://github.com/tkseneee/Classification-of-Heart-Sound
%cd Classification-of-Heart-Sound

Cloning into 'Classification-of-Heart-Sound'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), done.
/content/database/Classification-of-Heart-Sound


Importing the Database from Google Drive:

First add this Files to your Google Drive by pressing this Icon via Bowser:


https://drive.google.com/open?id=13ehWqXt8YDrmmjQc7XAUqcCk6Dwb69hy


<img src="https://i.stack.imgur.com/TqOfU.png" width="300" />

Connect ot your Google Drvie by this codes:
```
from google.colab import drive
drive.mount('/content/drive')
```


In [62]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/Data.rar /content

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Unrar the Data.rar file:

In [63]:
!sudo apt-get install unrar
%cd /content
!mkdir database
%cd database
!unrar e   /content/Data.rar  /content/database

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.5.8-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
/content
mkdir: cannot create directory ‘database’: File exists
/content/database

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Data.rar


Would you like to replace the existing file /content/database/set_a.csv
  7031 bytes, modified on 2019-03-19 09:33
with a new one
  7031 bytes, modified on 2019-03-19 09:33

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit q

Program aborted


### Now we have database files as you can see below :

So the Input Direction woule be : `INPUT_DIR='/content/database'`

<img src="https://i.stack.imgur.com/tG1an.png" width="300" />



In [64]:
!unzip -od set_a /content/database/set_a.zip
!unzip -od set_b /content/database/set_b.zip
!ls

Archive:  /content/database/set_a.zip
  inflating: set_a/artifact__201012172012.wav  
  inflating: set_a/artifact__201105040918.wav  
  inflating: set_a/artifact__201105041959.wav  
  inflating: set_a/artifact__201105051017.wav  
  inflating: set_a/artifact__201105060108.wav  
  inflating: set_a/artifact__201105061143.wav  
  inflating: set_a/artifact__201105190800.wav  
  inflating: set_a/artifact__201105280851.wav  
  inflating: set_a/artifact__201106010559.wav  
  inflating: set_a/artifact__201106010602.wav  
  inflating: set_a/artifact__201106021541.wav  
  inflating: set_a/artifact__201106030612.wav  
  inflating: set_a/artifact__201106031558.wav  
  inflating: set_a/artifact__201106040722.wav  
  inflating: set_a/artifact__201106040933.wav  
  inflating: set_a/artifact__201106040947.wav  
  inflating: set_a/artifact__201106041452.wav  
  inflating: set_a/artifact__201106050353.wav  
  inflating: set_a/artifact__201106061233.wav  
  inflating: set_a/artifact__201106070537.wav  
  

In [65]:
import numpy as np
import pandas as pd
import glob
import os
import librosa

In [66]:
INPUT_DIR="/content/database/"  # comment "C:\\Users\\senthilku\\Desktop\\Heart Sound"
SAMPLE_RATE = 16000
MAX_SOUND_CLIP_DURATION=12 #sec  

In [67]:
set_a=pd.read_csv(INPUT_DIR+"/set_a.csv")
#set_a_timing=pd.read_csv(INPUT_DIR+"/set_a_timing.csv")
set_b=pd.read_csv(INPUT_DIR+"/set_b.csv")
frames = [set_a, set_b]
data_ab=pd.concat(frames)

In [229]:
def audio_norm(data):
    max_data = np.max(data)
    min_data = np.min(data)
    data = (data-min_data)/(max_data-min_data+0.0001)
    return data-0.5

# get audio data without padding highest quality audio
def load_file_data_without_change(folder,file_names, duration=3, sr=16000):
    input_length=sr*duration
    # function to load files and extract features
    # file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("load file ",sound_file)
            # use kaiser_fast technique for faster extraction
            X, sr = librosa.load( sound_file, res_type='kaiser_fast') 
            dur = librosa.get_duration(y=X, sr=sr)
            # extract normalized mfcc feature from data
            #mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T,axis=0) 
        except Exception as e:
            print("Error encountered while parsing file: ", file)
        #feature = np.array(mfccs).reshape([-1,1])
        data.append(X)
    return data


# get audio data with a fix padding may also chop off some file
def load_file_data (folder,file_names, duration=12, sr=16000):
    input_length=sr*duration
    # function to load files and extract features
    # file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("load file ",sound_file)
            # use kaiser_fast technique for faster extraction
            X, sr = librosa.load( sound_file, sr=sr, duration=duration,res_type='kaiser_fast')
            # print ('X is :   '+str(X))
            dur = librosa.get_duration(y=X, sr=sr)
            y1=X
            # pad audio file same duration
            if (round(dur) < duration):
                print ("fixing audio lenght :", file_name)
                y1 = librosa.util.fix_length(X, input_length) 
            else:
              y1=X    
            y1 = audio_norm(y1)            
            # extract normalized mfcc feature from data
            #mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T,axis=0)             
        except Exception as e:
            print("Error encountered while parsing file: ", file)        
        #feature = np.array(mfccs).reshape([-1,1])
        data.append(y1)
    return data

In [69]:
# simple encoding of categories, limited to 3 types
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Map label text to integer
CLASSES = ['artifact','murmur','normal']
# {'artifact': 0, 'murmur': 1, 'normal': 3}
NB_CLASSES=len(CLASSES)

# Map integer value to text labels
label_to_int = {k:v for v,k in enumerate(CLASSES)}
print (label_to_int)
print (" ")
# map integer to label text
int_to_label = {v:k for k,v in label_to_int.items()}
print(int_to_label)

{'artifact': 0, 'murmur': 1, 'normal': 2}
 
{0: 'artifact', 1: 'murmur', 2: 'normal'}


In [ ]:
# load dataset-a, keep them separate for testing purpose
import os, fnmatch

A_folder=INPUT_DIR+'/set_a/'
# set-a
A_artifact_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'artifact*.wav')
A_artifact_sounds = load_file_data(folder=A_folder,file_names=A_artifact_files, duration=MAX_SOUND_CLIP_DURATION)
A_artifact_labels = [0 for items in A_artifact_files]

A_normal_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'normal*.wav')
A_normal_sounds = load_file_data(folder=A_folder,file_names=A_normal_files, duration=MAX_SOUND_CLIP_DURATION)
A_normal_labels = [2 for items in A_normal_sounds]

A_extrahls_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'extrahls*.wav')
A_extrahls_sounds = load_file_data(folder=A_folder,file_names=A_extrahls_files, duration=MAX_SOUND_CLIP_DURATION)
A_extrahls_labels = [1 for items in A_extrahls_sounds]

A_murmur_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'murmur*.wav')
A_murmur_sounds = load_file_data(folder=A_folder,file_names=A_murmur_files, duration=MAX_SOUND_CLIP_DURATION)
A_murmur_labels = [1 for items in A_murmur_files]

# test files
A_unlabelledtest_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a'), 'Aunlabelledtest*.wav')
A_unlabelledtest_sounds = load_file_data(folder=A_folder,file_names=A_unlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
A_unlabelledtest_labels = [-1 for items in A_unlabelledtest_sounds]

print ("loaded dataset-a")

In [71]:
# !find '/content/database/set_b' -name 'murmur*.wav' 

In [230]:
import os, fnmatch

# load dataset-b, keep them separate for testing purpose 
B_folder=INPUT_DIR+'/set_b/'
# set-b
B_normal_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b'), 'normal*.wav')  # include noisy files
B_normal_sounds = load_file_data(folder=B_folder,file_names=B_normal_files, duration=MAX_SOUND_CLIP_DURATION)
B_normal_labels = [2 for items in B_normal_sounds]

B_murmur_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b'), 'murmur*.wav')
B_murmur_sounds = load_file_data(folder=B_folder,file_names=B_murmur_files, duration=MAX_SOUND_CLIP_DURATION)
B_murmur_labels = [1 for items in B_murmur_files]

# B_murmur_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b'), 'murmur*.wav')  # include noisy files
# B_murmur_sounds = load_file_data(folder=B_folder,file_names=B_murmur_files, duration=MAX_SOUND_CLIP_DURATION)
# B_murmur_labels = [1 for items in B_murmur_files]

B_extrastole_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b'), 'extrastole*.wav')
B_extrastole_sounds = load_file_data(folder=B_folder,file_names=B_extrastole_files, duration=MAX_SOUND_CLIP_DURATION)
B_extrastole_labels = [1 for items in B_extrastole_files]

#test files
B_unlabelledtest_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b'), 'Bunlabelledtest*.wav')
B_unlabelledtest_sounds = load_file_data(folder=B_folder,file_names=B_unlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
B_unlabelledtest_labels = [-1 for items in B_unlabelledtest_sounds]
print ("loaded dataset-b")

load file  /content/database//set_b/normal__106_1306776721273_B1.wav
fixing audio lenght : normal__106_1306776721273_B1.wav
load file  /content/database//set_b/normal_noisynormal_141_1306520154450_B1.wav
fixing audio lenght : normal_noisynormal_141_1306520154450_B1.wav
load file  /content/database//set_b/normal__260_1309353164458_C.wav
fixing audio lenght : normal__260_1309353164458_C.wav
load file  /content/database//set_b/normal__170_1307970562729_A.wav
fixing audio lenght : normal__170_1307970562729_A.wav
load file  /content/database//set_b/normal__147_1306523973811_A.wav
fixing audio lenght : normal__147_1306523973811_A.wav
load file  /content/database//set_b/normal__137_1306764999211_D.wav
fixing audio lenght : normal__137_1306764999211_D.wav
load file  /content/database//set_b/normal__115_1306259437619_C.wav
fixing audio lenght : normal__115_1306259437619_C.wav
load file  /content/database//set_b/normal__153_1306848820671_A.wav
fixing audio lenght : normal__153_1306848820671_A.wa

## Question about the below error:

>ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

https://stackoverflow.com/questions/65067109/nested-numpy-array-concatenate-problem-array-all-the-input-arrays-must-have-sam


In [266]:
import numpy
import numpy as np 
a = np.array(A_artifact_sounds)
b= np.array(B_normal_sounds)#.reshape(-1,1
x_data3=numpy.concatenate((a, b))

ValueError: ignored

In [269]:
x_data1 = np.concatenate((A_artifact_sounds, A_normal_sounds,A_extrahls_sounds,A_murmur_sounds))

x_data2 = np.concatenate((B_normal_sounds,B_murmur_sounds,B_extrastole_sounds))

# x_data3=np.concatenate((np.reshape(x_data1, (-1, 1)), np.reshape(x_data2, (-1, 1))))
# x_data3=np.row_stack((x_data1,x_data2)).shape

print ('\n',np.shape(x_data1),np.shape(x_data2),np.shape(x_data3))
xx=np.array(x_data3[:,0])
print ('xx\n',np.shape(xx))
print (xx)
xd=np.reshape(xx, (-1, 1))
print ('xd\n',np.shape(xd))
print (xd)
# print (x_data3[:,0])

x_data3= np.concatenate((x_data1, x_data2))



 (124, 192000) (461,) (23808461, 1)
xx
 (23808461,)
[-0.07650777697563171 -0.09730669856071472 -0.05172833800315857 ...
 array([0.00871342, 0.01009369, 0.01195222, ..., 0.00930035, 0.00930035,
       0.00930035], dtype=float32)
 array([ 0.02457428,  0.0292967 ,  0.03165859, ..., -0.0067268 ,
       -0.0067268 , -0.0067268 ], dtype=float32)
 array([-0.06128126, -0.06926042, -0.07240129, ..., -0.0075101 ,
       -0.0060946 , -0.00427625], dtype=float32)]
xd
 (23808461, 1)
[[-0.07650777697563171]
 [-0.09730669856071472]
 [-0.05172833800315857]
 ...
 [array([0.00871342, 0.01009369, 0.01195222, ..., 0.00930035, 0.00930035,
       0.00930035], dtype=float32)]
 [array([ 0.02457428,  0.0292967 ,  0.03165859, ..., -0.0067268 ,
       -0.0067268 , -0.0067268 ], dtype=float32)]
 [array([-0.06128126, -0.06926042, -0.07240129, ..., -0.0075101 ,
       -0.0060946 , -0.00427625], dtype=float32)]]


ValueError: ignored

In [84]:
#combine set-a and set-b 
x_data = np.concatenate((A_artifact_sounds, A_normal_sounds,A_extrahls_sounds,A_murmur_sounds, 
                         B_normal_sounds,B_murmur_sounds,B_extrastole_sounds))

y_data = np.concatenate((A_artifact_labels, A_normal_labels,A_extrahls_labels,A_murmur_labels,
                         B_normal_labels,B_murmur_labels,B_extrastole_labels))

test_x = np.concatenate((A_unlabelledtest_sounds,B_unlabelledtest_sounds))
test_y = np.concatenate((A_unlabelledtest_labels,B_unlabelledtest_labels))

print ("combined training data record: ",len(y_data), len(test_y))

ValueError: ignored

In [ ]:
y_data.shape

In [ ]:
seed = 1000
# split data into Train, Validation and Test
x_train, x_test, y_train, y_test= train_test_split(x_data, y_data, train_size=0.8, random_state=seed, shuffle=True)
#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=seed, shuffle=True)

In [ ]:
x_test.shape

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, GlobalAvgPool1D, Dropout, BatchNormalization, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.utils import np_utils
from keras.regularizers import l2

In [ ]:
from scipy.signal import decimate
x_train = decimate(x_train, 8, axis=1, zero_phase=True)
x_train = decimate(x_train, 4, axis=1, zero_phase=True)
#x_train = decimate(x_train, 4, axis=1, zero_phase=True)
x_test = decimate(x_test, 8, axis=1, zero_phase=True)
x_test = decimate(x_test, 4, axis=1, zero_phase=True)
#x_test = decimate(x_test, 4, axis=1, zero_phase=True)

#Scale each observation to unit variance, it should already have mean close to zero.
#x_train = x_train / np.std(x_train, axis=1).reshape(-1,1)
#x_test = x_test / np.std(x_test, axis=1).reshape(-1,1)

x_train = x_train[:,:,np.newaxis]
x_test = x_test[:,:,np.newaxis]

In [ ]:
x_train.shape

In [ ]:
#new_labels = np.array(new_labels, dtype='int')
Y_train = np_utils.to_categorical(y_train)
Y_test=np_utils.to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=4, kernel_size=9, activation='relu', input_shape = x_train.shape[1:],kernel_regularizer = l2(0.025)))
model.add(MaxPool1D(strides=4))
model.add(BatchNormalization())
model.add(Conv1D(filters=4, kernel_size=(9), activation='relu',
                kernel_regularizer = l2(0.05)))
model.add(MaxPool1D(strides=4))
model.add(BatchNormalization())
model.add(Conv1D(filters=8, kernel_size=(9), activation='relu',
                 kernel_regularizer = l2(0.1)))
model.add(MaxPool1D(strides=4))
model.add(BatchNormalization())
model.add(Conv1D(filters=16, kernel_size=(9), activation='relu'))
model.add(MaxPool1D(strides=4))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv1D(filters=64, kernel_size=(4), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=(1), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.75))
model.add(GlobalAvgPool1D())
model.add(Dense(3, activation='softmax'))

In [ ]:
def batch_generator(x_train, y_train, batch_size):
    """
    Rotates the time series randomly in time
    """
    x_batch = np.empty((batch_size, x_train.shape[1], x_train.shape[2]), dtype='float32')
    y_batch = np.empty((batch_size, y_train.shape[1]), dtype='float32')
    full_idx = range(x_train.shape[0])
    
    while True:
        batch_idx = np.random.choice(full_idx, batch_size)
        x_batch = x_train[batch_idx]
        y_batch = y_train[batch_idx]
    
        for i in range(batch_size):
            sz = np.random.randint(x_batch.shape[1])
            x_batch[i] = np.roll(x_batch[i], sz, axis = 0)
     
        yield x_batch, y_batch
        
weight_saver = ModelCheckpoint('set_a_weights.h5', monitor='val_loss', 
                               save_best_only=True, save_weights_only=True)

In [ ]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.8**x)

In [ ]:
x_train.shape

In [ ]:
hist = model.fit_generator(batch_generator(x_train, Y_train, 8),
                   epochs=10, steps_per_epoch=1000,
                   validation_data=(x_test, Y_test),
                   callbacks=[weight_saver, annealer],
                   verbose=2)

In [ ]:
model.load_weights('set_a_weights.h5')
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()